In [30]:
%matplotlib inline
import pandas as pd
import jieba
import numpy as np
def jieba_lines(lines):
    cut_lines = []
    for line in lines:
        cut_line = jieba.lcut(line)
        cut_lines.append(cut_line)
    return cut_lines
jieba.set_dictionary('big5_dict.txt')
jieba.load_userdict('my_dict.txt')

Building prefix dict from C:\Users\Administrator\Desktop\DL\Comp1\big5_dict.txt ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.u4db63a6c4a8ce99b1489a4d22431124c.cache
Loading model cost 1.596 seconds.
Prefix dict has been built succesfully.


In [46]:
cut_programs = np.load('cut_Programs.npy')

## Prepare question

In [31]:
cut_questions = []
n = len(questions)

for i in range(n):
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    cut_question.append(jieba_lines(lines))
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line))
    
    cut_questions.append(cut_question)

In [62]:
print("%d questions" % len(cut_questions))
print(len(cut_questions[0]))
# 2 4
n = 7
# 1 question
print(cut_questions[n][0])

# 6 optional reponses
for i in range(1, 7):
    print(cut_questions[n][i])

500 questions
7
[['輝哥', ' ', '全部', '都', '被', '你', '吃', '了', ' ', '你', '很', '過分'], ['就', '兩顆', ' ', '你還', '剩下', '這樣子', ' ', '那', '我', '吃', '什麼'], ['沒有', ' ', '因為', '你們', '一直', '在問', '問題', ' ', '我', '閒著', '無聊'], []]
['然後', '煮', '一', '煮', '就', '變成', '紅色', '的', '嗎']
['我', '就', '把', '它', '吃完', '了']
['現在', '要', '把', '那個', '木頭', '給', '粉碎', '掉']
['因為', '聽說', '以前', '是', '用燒', '的']
['會', '造成', '空氣', '污染']
['像', '他們', '現在', '這樣', '粉碎', '掉']


In [ ]:
# np.save('cut_Programs.npy', cut_programs)
np.save('cut_Questions.npy', cut_questions)

## Preprocessing: Word Dictionary & Out-of-Vocabulary

In [66]:
import operator

word_dict = dict()
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1
for program in cut_programs:
    for lines in program:
        for line in lines:
            for w in line:
                add_word_dict(w)
for question in cut_questions:
    lines = question[0]
    for line in lines:
        for w in line:
            add_word_dict(w)
    
    for i in range(1, 7):
        line = question[i]
        for w in line:
            add_word_dict(w)

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
print("Total %d words in word_dict" % len(word_dict))

Total 174424 words in word_dict


In [81]:
VOC_START = 9
VOC_SIZE = 15000
voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
print(voc_dict[:10])
print()
print("Total %d words in voc_dict" % len(voc_dict))
np.save('voc_dict.npy', voc_dict)
voc_dict = np.load('voc_dict.npy')

[('有', 119784), ('不', 118402), ('在', 117590), ('對', 111662), ('啊', 107267), ('...', 107010), ('要', 97042), ('好', 91516), ('這個', 90986), ('都', 90949)]

Total 15000 words in voc_dict


In [86]:
print(cut_programs[0][0][20])

['而且', '很', '喜歡', '搞笑', '的', '阿輝']


## Preprocessing: Generating Training Data

In [54]:
import random

NUM_TRAIN = 10000
NUM_PROGRAM = 8
TRAIN_VALID_RATE = 0.7
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            program_id = random.randint(0, NUM_PROGRAM-1)
            episode_id = random.randint(0, len(cut_programs[program_id])-1)
            line_id = random.randint(0, len(cut_programs[program_id][episode_id])-2)
            
            Xs.append([cut_programs[program_id][episode_id][line_id], 
                       cut_programs[program_id][episode_id][line_id+1]])
            Ys.append(1)
            
        else:
            first_program_id = random.randint(0, NUM_PROGRAM-1)
            first_episode_id = random.randint(0, len(cut_programs[first_program_id])-1)
            first_line_id = random.randint(0, len(cut_programs[first_program_id][first_episode_id])-1)
            
            second_program_id = random.randint(0, NUM_PROGRAM-1)
            second_episode_id = random.randint(0, len(cut_programs[second_program_id])-1)
            second_line_id = random.randint(0, len(cut_programs[second_program_id][second_episode_id])-1)
            
            Xs.append([cut_programs[first_program_id][first_episode_id][first_line_id], 
                       cut_programs[second_program_id][second_episode_id][second_line_id]])
            Ys.append(0)
    
    return Xs, Ys
Xs, Ys = generate_training_data()

x_train, y_train = Xs[:int(NUM_TRAIN*TRAIN_VALID_RATE)], Ys[:int(NUM_TRAIN*TRAIN_VALID_RATE)]
x_valid, y_valid = Xs[int(NUM_TRAIN*TRAIN_VALID_RATE):], Ys[int(NUM_TRAIN*TRAIN_VALID_RATE):]

In [106]:
def greedy_gen():
    Xs, Ys = [], []
    all_pos = 0
    all_neg = 0
    for ep in cut_programs:
        for lines in ep:
            for line_id in range(len(lines)-1): 
                Xs.append([lines[line_id], 
                           lines[line_id+1]])
                Ys.append(1)
                all_pos+=1
    print("p items:",all_pos)
####################
    for i in range(all_pos):
        first_program_id = random.randint(0, NUM_PROGRAM-1)
        first_episode_id = random.randint(0, len(cut_programs[first_program_id])-1)
        first_line_id = random.randint(0, len(cut_programs[first_program_id][first_episode_id])-1)

        second_program_id = random.randint(0, NUM_PROGRAM-1)
        second_episode_id = random.randint(0, len(cut_programs[second_program_id])-1)
        second_line_id = random.randint(0, len(cut_programs[second_program_id][second_episode_id])-1)

        Xs.append([cut_programs[first_program_id][first_episode_id][first_line_id], 
                   cut_programs[second_program_id][second_episode_id][second_line_id]])
        Ys.append(0)
        all_neg+=1
    print("p+n items:",all_pos+all_neg)
    return Xs, Ys
# Xs, Ys = greedy_gen()
# x_train, y_train = Xs[:int(NUM_TRAIN*TRAIN_VALID_RATE)], Ys[:int(NUM_TRAIN*TRAIN_VALID_RATE)]
# x_valid, y_valid = Xs[int(NUM_TRAIN*TRAIN_VALID_RATE):], Ys[int(NUM_TRAIN*TRAIN_VALID_RATE):]

p items: 2394065
p+n items: 2394065


In [109]:
np.save('Xs.npy', Xs)
np.save('Ys.npy', Ys)


In [93]:
print(Xs[1])
print(cut_programs[0][0][0])

[['我', '覺得', '心是', '很', '重要', '的'], ['直接', '進入', '你', '腦部']]
['還好', '天氣', '不錯']


In [80]:
example_doc = []

# lines in 1st episode in program 0 
for line in cut_programs[0][0]:
    example_line = ''
    for w in line:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print(example_doc[:10])

['還好 天氣 不錯 ', '昨天 晚上 ', '看到 很多 流星 ', '這次 收穫 真 豐富 ', '當然 豐富 啦 ', '說 嘛 ', '精心 製作 ', '被 一個 人 吃掉 ', '真的 嗎 ', '不要 忘記 要 做 秘密 檔案 ']


## Word2Vec: BoW (Bag-Of-Words)

In [72]:
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer

# ngram_range=(min, max), default: 1-gram => (1, 1)
count = CountVectorizer(ngram_range=(1, 1))

count.fit(example_doc)
BoW = count.vocabulary_
print('[vocabulary]\n')
for key in list(BoW.keys())[:10]:
    print('%s %d' % (key, BoW[key]))

[vocabulary]

還好 469
天氣 168
不錯 46
昨天 268
晚上 270
看到 352
很多 217
流星 310
這次 456
收穫 259
